
### Web-Scraping-and-Data-Analysis project-on-Economic-impact-of-COVID-19 

### Importing the required libraries

In [1]:
import requests
from bs4 import BeautifulSoup as bs
from functools import reduce
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
import datetime
import yfinance as yf

### Covid cases

In [2]:
# URL for Covid cases from wikipedia

urlcovid = "https://en.wikipedia.org/w/index.php?title=Template:COVID-19_pandemic_data/India_medical_cases_chart&action=edit"
response_covid = requests.get(urlcovid)
response_covid

<Response [200]>

In [3]:
# Scrapping data from URL

response_covid = requests.get(urlcovid)
response_covid.text

# Data cleaning to get data from text

soup_covid = bs(response_covid.text)
covid_find = soup_covid.find("textarea",class_="mw-editfont-monospace").text.split("\n")

cases = []
for i in covid_find:
    cases.append(i.split(";"))
cases = cases[17:]

# Making a Data frame

cov_cases = pd.DataFrame(cases)
df0=pd.DataFrame()
df0["date"]= cov_cases[0][:524]
df0["cases"]= cov_cases[3][:524]

# Replacing missing values with NaN and droping them

df0.replace(r'', np.nan,inplace = True)
df0= df0.dropna().reset_index()
df0.reset_index(drop=True)

,index,date,cases
0,0,2020-01-30,1
1,2,2020-02-02,2
2,3,2020-02-03,3
3,5,2020-02-21,3
4,7,2020-03-02,5
...,...,...,...
516,519,2021-07-27,31440492
517,520,2021-07-28,31483463
518,521,2021-07-29,31526628
519,522,2021-07-30,31571299


In [4]:
# Filling zeros where no cases are present

fill1 = []
for i in range(244):
    fill1.append(0)
    
# Filling the dates where cases are present

fill2 =[1,0,0,2,3]
fill3 = []
for i in range(17):
    fill3.append(0)
fill4 = []
for i in range(9):
    fill4.append(0)
covid_cases =fill1+fill2+fill3+fill4
covid_cases =pd.DataFrame(covid_cases)
covid_cases.reset_index(drop=True, inplace=True)
covid_cases

# Concatenating with previous data frame

df = df0.iloc[4:]
cf = df["cases"]
cf.reset_index()
cov_df = pd.concat([covid_cases,cf]) 
cov_df["Covid cases"]=cov_df
cov_df.drop(0, inplace=True, axis=1)
new_df = cov_df.reset_index()
covid = pd.DataFrame(new_df["Covid cases"])
covid["Covid cases"] = covid["Covid cases"].astype(int)
covid

,Covid cases
0,0
1,0
2,0
3,0
4,0
...,...
787,31440492
788,31483463
789,31526628
790,31571299


### Currency exchange rates

In [5]:
# Scrapping data from url

rates =[]
for i in range(2019,2022):
    my_url = f"https://www.exchangerates.org.uk/USD-INR-spot-exchange-rates-history-{i}.html"
    response = requests.get(my_url)
    soup_rupee = bs(response.text)
    rupee_find = soup_rupee.find("div",class_="row").text.split("US Dollar ")
    rates.append(rupee_find)
    
# data cleaning

split_rates = []
for i in range(len(rates)):
        temp = re.findall(r"[₹]\d+.\d+",str(rates[i]))
        split_rates.append(temp)
exchange_rates =[]
for i in split_rates:
    for j in i:
        temp = j.removeprefix("₹")
        exchange_rates.append(temp)
# Finding indexes of required dates in raw text for slicing

#'to Indian Rupee Monthly Exchange Rates\n\nSaturday  1 June 2019$1 USD = ₹69.5795'
#'Indian Rupee rate for 30/07/2021Saturday 31 July 2021$1 USD = ₹74.3793'

index_june2019 = exchange_rates.index("69.5795")
index_july2021 = exchange_rates.index("74.3793")
exchange_rates = exchange_rates[index_june2019:index_july2021+1]
exchange_rates = np.array(exchange_rates,dtype=float)
exchange_rates

array([69.5795, 69.5795, 69.5781, 69.1697, 69.3155, 69.4006, 69.1263,
       69.39  , 69.39  , 69.3532, 69.5114, 69.3975, 69.416 , 69.5591,
       69.8798, 69.8789, 69.7813, 69.8784, 69.4808, 69.6089, 69.5201,
       69.58  , 69.58  , 69.5777, 69.2623, 69.3316, 69.2464, 69.0564,
       68.9507, 68.9507, 68.9499, 68.9048, 68.8455, 68.7993, 68.5488,
       68.4503, 68.4499, 68.45  , 68.6229, 68.5491, 68.374 , 68.4718,
       68.5604, 68.5604, 68.5559, 68.5368, 68.6731, 68.796 , 68.7339,
       68.9371, 68.9355, 68.8378, 68.93  , 69.0293, 68.981 , 69.1406,
       68.8603, 68.8603, 68.88  , 68.7302, 68.8499, 69.009 , 69.3056,
       69.667 , 69.667 , 69.6653, 71.128 , 70.9137, 71.1905, 70.4168,
       71.0423, 71.0423, 71.0455, 71.2815, 71.1242, 71.6189, 71.4223,
       71.143 , 71.143 , 71.1442, 71.6763, 71.4833, 71.4451, 71.9293,
       71.8255, 71.8196, 71.509 , 71.9559, 71.5472, 71.8259, 71.691 ,
       71.74  , 71.74  , 71.7405, 71.9725, 72.1471, 71.9798, 71.9332,
       71.6771, 71.6

### Crude oil prices

In [6]:
# Scrapping data from URL

response_oilrates = requests.get("https://www.indexmundi.com/commodities/?commodity=crude-oil&months=60&currency=inr")

#Response_oilrates.text

soup_oilrates = bs(response_oilrates.text)
crudeoil = soup_oilrates.find("div",class_="row").text.split("\n")
find_index_start = crudeoil.index('Jun 20194,149.67-11.03%')
find_index_stop =  crudeoil.index('Jul 20215,460.903.40%') + 1
crudeoil = crudeoil[find_index_start : find_index_stop]

#Finding the text and cleaning

find_price = []
crudeoil_monthlyrates = []

for i in crudeoil:
        temp = re.findall(r"\d+.\d+.\d+|\d+-\d+.\d+",i)
        find_price.append(temp)
    

# Flattening the list

find_price = sum(find_price,[])

for i in find_price:
        temp = float(i[4:].replace(",",""))
        crudeoil_monthlyrates.append(temp)
crudeoil_monthlyrates

# Approximating minimum and maximum values for the average values

crudeoilprice_minrange = []
crudeoilprice_maxrange = []
for i in crudeoil_monthlyrates:
    min_range = i - 30
    max_range = i + 30
    crudeoilprice_minrange.append(min_range)
    crudeoilprice_maxrange.append(max_range)


In [7]:
# create lists to pass in the function

years = [2019,2020,2021]
common_year = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
leap_year = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
dates = []
for i in years:
    if i % 4 == 0:
        for j in leap_year:
            dates.append(j)
    else:
        for k in common_year:
            dates.append(k)
dates = dates[5:-5]
len(dates)

26

In [8]:
#function to create random daily values

ranlist = []
def month_days(x,y,n):
    global random_float_list
    for i in range(0,n):
        k = round(random.uniform(x,y),2)
        ranlist.append(k)

In [9]:
for x,y,z in zip(crudeoilprice_minrange,crudeoilprice_maxrange,dates):
    month_days(x,y,z)
crude_oil_rates = ranlist
crude_oil_rates

[4155.66,
 4151.26,
 4159.23,
 4132.73,
 4143.73,
 4123.98,
 4171.98,
 4144.75,
 4171.29,
 4125.79,
 4126.82,
 4134.98,
 4170.47,
 4126.73,
 4157.93,
 4144.67,
 4133.36,
 4127.98,
 4140.19,
 4133.28,
 4128.2,
 4177.52,
 4137.09,
 4124.58,
 4148.08,
 4123.28,
 4169.48,
 4167.76,
 4135.15,
 4156.61,
 4242.28,
 4253.66,
 4202.02,
 4235.98,
 4250.57,
 4221.73,
 4231.57,
 4204.45,
 4246.07,
 4218.86,
 4216.83,
 4233.89,
 4245.09,
 4251.15,
 4205.78,
 4222.78,
 4242.64,
 4251.04,
 4258.26,
 4223.47,
 4226.63,
 4245.36,
 4217.87,
 4254.58,
 4234.63,
 4257.51,
 4208.57,
 4223.65,
 4237.95,
 4249.27,
 4227.31,
 4085.13,
 4107.56,
 4114.04,
 4118.74,
 4094.71,
 4110.04,
 4087.29,
 4076.35,
 4091.31,
 4123.8,
 4131.86,
 4080.27,
 4101.19,
 4099.05,
 4091.02,
 4084.36,
 4098.15,
 4125.06,
 4089.24,
 4129.84,
 4128.49,
 4114.24,
 4126.37,
 4096.54,
 4105.41,
 4121.96,
 4100.17,
 4117.8,
 4107.2,
 4108.92,
 4098.87,
 4287.64,
 4311.67,
 4301.29,
 4293.86,
 4272.58,
 4302.84,
 4280.8,
 4301.41,
 4280

### Gold prices

In [10]:
# Scrapping data from URL

response_gold =requests.get("https://www.indexmundi.com/commodities/?commodity=gold&months=60&currency=inr")
soup_gold = bs(response_gold.text)
gold = soup_gold.find("div",class_="row").text.split("\n")
find_goldindex_start = gold.index('Jun 201994,370.205.34%')
find_goldindex_stop =  gold.index('Jul 2021134,722.00-0.17%') + 1
gold_index = gold[find_goldindex_start : find_goldindex_stop]
gold_price = []
gold_monthlyrates = []
for i in gold_index:
    g1 = re.findall(r"\d+.\d+.\d+",i)
    gold_price.append(g1)

# flattening the list

gold_price = sum(gold_price,[])

for i in gold_price:
        temp = float(i[4:].replace(",",""))
        gold_monthlyrates.append(temp)
        
# approximating minimum and maximum values for the average values

gold_minrange = []
gold_maxrange = []
for i in gold_monthlyrates:
    min_range = i - 30
    max_range = i + 30
    gold_minrange.append(min_range)
    gold_maxrange.append(max_range)

len(gold_monthlyrates)

26

In [11]:
goldlist = []
def month_days(x,y,n):
    global random_float_list
    for i in range(0,n):
        z = round(random.uniform(x,y),2)
        goldlist.append(z)

In [12]:
for x,y,z in zip(gold_minrange,gold_maxrange,dates):
    month_days(x,y,z)
gold_price = goldlist
gold_price

[94394.97,
 94376.54,
 94356.74,
 94370.67,
 94385.42,
 94365.93,
 94359.89,
 94399.85,
 94370.59,
 94347.78,
 94345.52,
 94384.27,
 94380.92,
 94377.51,
 94366.75,
 94340.65,
 94363.15,
 94350.83,
 94381.89,
 94388.97,
 94356.32,
 94366.05,
 94348.35,
 94395.4,
 94359.16,
 94389.18,
 94370.16,
 94378.21,
 94382.98,
 94354.32,
 97194.19,
 97242.67,
 97196.28,
 97232.82,
 97190.63,
 97188.82,
 97238.35,
 97245.13,
 97186.06,
 97191.17,
 97187.02,
 97234.86,
 97231.0,
 97233.69,
 97227.65,
 97228.79,
 97219.83,
 97237.24,
 97245.33,
 97238.11,
 97198.92,
 97222.75,
 97239.09,
 97243.96,
 97223.27,
 97239.42,
 97213.09,
 97234.16,
 97197.82,
 97200.94,
 97198.77,
 106720.64,
 106741.53,
 106724.72,
 106777.14,
 106769.14,
 106734.03,
 106764.6,
 106727.21,
 106769.48,
 106730.19,
 106752.97,
 106745.21,
 106764.6,
 106763.01,
 106724.44,
 106754.73,
 106724.23,
 106734.08,
 106722.36,
 106766.98,
 106758.87,
 106742.93,
 106726.76,
 106722.57,
 106720.79,
 106720.96,
 106751.22,
 106758.0

### Food prices

In [13]:
# Scrapping data from URL

response_food = requests.get("https://www.indexmundi.com/commodities/?commodity=food-price-index&months=60")
soup_food = bs(response_food.text)
food = soup_food.find("div",class_="row").text.split("\n")
find_foodindex_start = food.index('Jun 201988.063.63%')
find_foodindex_stop =  food.index('Jul 2021121.08-1.18%') + 1
food_index= food[find_foodindex_start:find_foodindex_stop]

food_price = []
food_monthlyrates = []

for i in food_index:
    m1 = re.findall(r"\d+.\d+.\d+|\d+-\d+.\d+",i)
    food_price.append(m1)

# Flattening the list

food_price = sum(food_price,[])

for i in food_price:
        temp = float(i[4:-3].replace(",",""))
        food_monthlyrates.append(temp)
        
# Approximating minimum and maximum values for the average values

food_minrange = []
food_maxrange = []
for i in food_monthlyrates:
    min_range = i - 30
    max_range = i + 30
    food_minrange.append(min_range)
    food_maxrange.append(max_range)
len(food_monthlyrates)

26

In [14]:
foodlist = []
def month_days(x,y,n):
    global random_float_list
    for i in range(0,n):
        z = round(random.uniform(x,y),2)
        foodlist.append(z)

In [15]:
for x,y,z in zip(food_minrange,food_maxrange,dates):
    month_days(x,y,z)
food_index = foodlist
food_index

[91.95,
 79.57,
 82.91,
 64.71,
 82.18,
 62.13,
 95.58,
 78.44,
 87.41,
 80.4,
 94.35,
 113.33,
 96.92,
 74.2,
 84.33,
 79.77,
 117.18,
 108.31,
 102.83,
 87.86,
 82.09,
 87.56,
 82.16,
 76.35,
 81.34,
 72.73,
 84.7,
 79.3,
 99.96,
 107.92,
 106.75,
 112.63,
 80.96,
 95.79,
 76.21,
 64.41,
 110.78,
 83.93,
 91.6,
 71.65,
 58.11,
 113.29,
 108.99,
 78.05,
 94.38,
 89.2,
 74.18,
 115.75,
 86.05,
 110.9,
 65.33,
 68.41,
 106.13,
 111.61,
 77.02,
 103.1,
 64.54,
 57.56,
 59.57,
 116.41,
 71.03,
 94.27,
 88.26,
 60.33,
 104.68,
 72.66,
 57.58,
 79.52,
 60.53,
 101.39,
 78.84,
 96.56,
 86.56,
 98.1,
 92.78,
 103.71,
 85.12,
 82.44,
 74.25,
 66.27,
 113.0,
 109.29,
 59.27,
 85.23,
 73.28,
 78.13,
 71.33,
 72.73,
 101.54,
 93.69,
 75.13,
 99.1,
 99.13,
 88.23,
 112.86,
 92.8,
 69.82,
 69.26,
 87.88,
 64.08,
 106.27,
 59.27,
 66.98,
 110.04,
 107.57,
 63.81,
 64.01,
 78.29,
 95.95,
 102.92,
 71.93,
 94.7,
 91.17,
 101.28,
 57.29,
 63.69,
 76.4,
 89.78,
 101.67,
 93.76,
 86.72,
 90.55,
 58.77,
 

### Getting Stock data using Yahoo finance API

### Nifty IT

In [16]:
# Set the start and end date

start_date = '2019-05-31'
end_date = '2021-08-03'

# Set the ticker(tickers can be found in yahoofinance website)

ticker = '^CNXIT'

# Get the data

it_date = yf.download(ticker, start_date, end_date)
itdf=pd.DataFrame(it_date)

#converted datetime into column

itdf.reset_index(inplace=True)
itdf["Date"]=itdf.iloc[:,0]
itdf.head()

#added missing dates with date range and imputed corresponding columns with previous date

daterange = pd.date_range(start=itdf["Date"].min(), end=itdf["Date"].max())
it = itdf.set_index('Date').reindex(daterange).fillna(0.0).rename_axis('Date').reset_index()
it.head()
it["NIFTY_IT"] = it.iloc[:,4]

#replaced zero with nan

it["NIFTY_IT"].replace(0, np.nan,inplace = True)

#replaced nan with previous value

it["NIFTY_IT"].fillna(method='pad', inplace=True)
it1 = it.iloc[1:793]
nifty_it =it1.iloc[:,0::7]
date =nifty_it.iloc[:,0]
nifty_it = nifty_it.reset_index(drop=True)
nifty_it

[*********************100%***********************]  1 of 1 completed


,Date,NIFTY_IT
0,2019-06-01,16160.650391
1,2019-06-02,16160.650391
2,2019-06-03,16360.549805
3,2019-06-04,16105.200195
4,2019-06-05,16105.200195
...,...,...
787,2021-07-27,29892.550781
788,2021-07-28,29954.150391
789,2021-07-29,30371.400391
790,2021-07-30,30480.050781


In [17]:
date = nifty_it.iloc[:,0].reset_index(drop=True)
date = pd.DataFrame(date)
date.head()

,Date
0,2019-06-01
1,2019-06-02
2,2019-06-03
3,2019-06-04
4,2019-06-05


### Nifty Bank

In [18]:
start_date = '2019-05-31'
end_date = '2021-08-03'
ticker = '^NSEBANK'
bank_date = yf.download(ticker, start_date, end_date)
bank_df=pd.DataFrame(bank_date)

#converted datetime into column

bank_df.reset_index(inplace=True)
bank_df["Date"]=bank_df.iloc[:,0]

#added missing dates with date range and imputed corresponding columns with previous date

bankrange = pd.date_range(start=bank_df["Date"].min(), end=bank_df["Date"].max())
bank_index = bank_df.set_index('Date').reindex(bankrange).fillna(0.0).rename_axis('Date').reset_index()
bank_index["NIFTY_BANK"] = bank_index.iloc[:,4]

#replaced zero with nan

bank_index["NIFTY_BANK"].replace(0, np.nan,inplace = True)

#replaced nan with previous value

bank_index["NIFTY_BANK"].fillna(method='pad', inplace=True)
bank = bank_index.iloc[1:793]
nifty_bankx =bank.iloc[:,0::7]
nifty_bank = nifty_bankx.iloc[:,1:]
nifty_bank = nifty_bank.reset_index(drop=True)
nifty_bank

[*********************100%***********************]  1 of 1 completed


,NIFTY_BANK
0,31375.400391
1,31375.400391
2,31653.650391
3,31589.050781
4,31589.050781
...,...
787,34797.449219
788,34532.898438
789,34691.500000
790,34584.351562


### Nifty 50

In [19]:
start_date = '2019-05-31'
end_date = '2021-08-03'
ticker = '^NSEI'
fifty_date = yf.download(ticker, start_date, end_date)
fifty_df =pd.DataFrame(fifty_date)

#converted datetime into column

fifty_df.reset_index(inplace=True)
fifty_df["Date"]=fifty_df.iloc[:,0]

#added missing dates with date range and imputed corresponding columns with previous date

fiftyrange = pd.date_range(start=fifty_df["Date"].min(), end=fifty_df["Date"].max())
fifty_index = fifty_df.set_index('Date').reindex(fiftyrange).fillna(0.0).rename_axis('Date').reset_index()
fifty_index["NIFTY_50"] = fifty_index.iloc[:,4]

#replaced zero with nan

fifty_index["NIFTY_50"].replace(0, np.nan,inplace = True)

#replaced nan with previous value

fifty_index["NIFTY_50"].fillna(method='pad', inplace=True)
fifty = fifty_index.iloc[1:793]
nifty_50X =fifty.iloc[:,0::7]
nifty_50 = nifty_50X.iloc[:,1:]
nifty_50 = nifty_50.reset_index(drop=True)
nifty_50

[*********************100%***********************]  1 of 1 completed


,NIFTY_50
0,11922.799805
1,11922.799805
2,12088.549805
3,12021.650391
4,12021.650391
...,...
787,15746.450195
788,15709.400391
789,15778.450195
790,15763.049805


## Final Dataframe

In [20]:
data = pd.DataFrame({"Date":date["Date"],"Covid cases":covid["Covid cases"],"USD( $ ) to INR( ₹ )":exchange_rates,
                      "crude oil ( ₹ / Barrel )":crude_oil_rates,
                      "Gold Price( ₹ / Troy ounce )":gold_price,
                      "Food Price Index":food_index,"NIFTY_IT":nifty_it["NIFTY_IT"],"NIFTY_BANK":nifty_bank["NIFTY_BANK"], "NIFTY_50":nifty_50["NIFTY_50"]
                      })
data

,Date,Covid cases,USD( $ ) to INR( ₹ ),crude oil ( ₹ / Barrel ),Gold Price( ₹ / Troy ounce ),Food Price Index,NIFTY_IT,NIFTY_BANK,NIFTY_50
0,2019-06-01,0,69.5795,4155.66,94394.97,91.95,16160.650391,31375.400391,11922.799805
1,2019-06-02,0,69.5795,4151.26,94376.54,79.57,16160.650391,31375.400391,11922.799805
2,2019-06-03,0,69.5781,4159.23,94356.74,82.91,16360.549805,31653.650391,12088.549805
3,2019-06-04,0,69.1697,4132.73,94370.67,64.71,16105.200195,31589.050781,12021.650391
4,2019-06-05,0,69.3155,4143.73,94385.42,82.18,16105.200195,31589.050781,12021.650391
...,...,...,...,...,...,...,...,...,...
787,2021-07-27,31440492,74.2838,5437.89,134732.84,143.98,29892.550781,34797.449219,15746.450195
788,2021-07-28,31483463,74.5378,5473.85,134744.56,102.48,29954.150391,34532.898438,15709.400391
789,2021-07-29,31526628,74.4524,5434.12,134726.85,143.84,30371.400391,34691.500000,15778.450195
790,2021-07-30,31571299,74.2468,5468.43,134718.59,121.78,30480.050781,34584.351562,15763.049805


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792 entries, 0 to 791
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Date                          792 non-null    datetime64[ns]
 1   Covid cases                   792 non-null    int32         
 2   USD( $ ) to INR( ₹ )          792 non-null    float64       
 3   crude oil ( ₹ / Barrel )      792 non-null    float64       
 4   Gold Price( ₹ / Troy ounce )  792 non-null    float64       
 5   Food Price Index              792 non-null    float64       
 6   NIFTY_IT                      792 non-null    float64       
 7   NIFTY_BANK                    792 non-null    float64       
 8   NIFTY_50                      792 non-null    float64       
dtypes: datetime64[ns](1), float64(7), int32(1)
memory usage: 52.7 KB


### Converting dataframe to csv file for further analysis

In [22]:
data.to_csv('webscraping-covid.csv')